In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os, csv, bz2, collections, datetime, sys
import geopandas as gpd
import numpy as np
import tilemapbase
import shapely.geometry
import pyproj
proj = pyproj.Proj({"init":"EPSG:2768"})

In [2]:
datadir = os.path.join("..", "..", "..", "..", "Data")
#datadir = os.path.join("/media", "disk", "Data")
filename = os.path.join(datadir, "SF_Police_Department_Incidents.csv.bz2")

def gen():
    with bz2.open(filename, "rt", encoding="UTF8") as f:
        reader = csv.reader(f)
        yield from reader

In [3]:
print(next(gen()))

['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId']


The fields:

- `IncidntNum` : Seems not to be quite unique.  The _vast_ majority of repeats are multiple offenses committed at the same time/place.  A rather small are linked crimes across different times (often stolen vehicle later recovered).
- `Category` : The crime type, from a small list (see below)
- `Descript` : For each `Category`, gives the sub type of the crime (_lots_ of choices!)
- `DayOfWeek` : "Monday" etc. agrees with `Date`
- `Date` : Month/Day/Year like "05/26/2017"
- `Time` : Standard 24 hour clock like "23:12" or "05:07"
- `PdDistrict` : Blank, or one of a small number of district names.
- `Resolution` : One of a small number of resolutions.
- `Address`
- `X`, `Y` : Longitude and Latitude.  A very small number of locations are false '(90, -120.5)'
- `Location` : Always "(Latitude, Longitude)"
- `PdId` : Unique ID

Check for repeats in `IncidntNum`: there are loads

In [8]:
rows = gen()
next(rows)
IncidntNum = [row[0] for row in rows]
len(IncidntNum), len(set(IncidntNum))

(2120044, 1673403)

But `PdId` is unique.

In [9]:
rows = gen()
next(rows)
PdId = [row[12] for row in rows]
len(PdId), len(set(PdId))

(2120044, 2120044)

Look at `Category` : "main crime type"

In [10]:
rows = gen()
next(rows)
set(row[1] for row in rows)

{'ARSON',
 'ASSAULT',
 'BAD CHECKS',
 'BRIBERY',
 'BURGLARY',
 'DISORDERLY CONDUCT',
 'DRIVING UNDER THE INFLUENCE',
 'DRUG/NARCOTIC',
 'DRUNKENNESS',
 'EMBEZZLEMENT',
 'EXTORTION',
 'FAMILY OFFENSES',
 'FORGERY/COUNTERFEITING',
 'FRAUD',
 'GAMBLING',
 'KIDNAPPING',
 'LARCENY/THEFT',
 'LIQUOR LAWS',
 'LOITERING',
 'MISSING PERSON',
 'NON-CRIMINAL',
 'OTHER OFFENSES',
 'PORNOGRAPHY/OBSCENE MAT',
 'PROSTITUTION',
 'RECOVERED VEHICLE',
 'ROBBERY',
 'RUNAWAY',
 'SECONDARY CODES',
 'SEX OFFENSES, FORCIBLE',
 'SEX OFFENSES, NON FORCIBLE',
 'STOLEN PROPERTY',
 'SUICIDE',
 'SUSPICIOUS OCC',
 'TREA',
 'TRESPASS',
 'VANDALISM',
 'VEHICLE THEFT',
 'WARRANTS',
 'WEAPON LAWS'}

Look at `Descript` and join to `Category`.  Lots of choices.

In [11]:
rows = gen()
next(rows)
types = collections.defaultdict(set)
for row in rows:
    types[row[1]].add(row[2])
types["ARSON"]

{'ARSON',
 'ARSON OF A COMMERCIAL BUILDING',
 'ARSON OF A POLICE BUILDING',
 'ARSON OF A POLICE VEHICLE',
 'ARSON OF A VACANT BUILDING',
 'ARSON OF A VEHICLE',
 'ARSON OF AN INHABITED DWELLING',
 'ARSON WITH GREAT BODILY INJURY',
 'ATTEMPTED ARSON',
 'FIRE, UNLAWFULLY CAUSING'}

In [12]:
for key in types:
    print(key, len(types[key]))

NON-CRIMINAL 40
ROBBERY 74
ASSAULT 64
SECONDARY CODES 9
VANDALISM 25
BURGLARY 57
LARCENY/THEFT 63
DRUG/NARCOTIC 75
WARRANTS 10
VEHICLE THEFT 18
OTHER OFFENSES 179
WEAPON LAWS 46
ARSON 10
MISSING PERSON 3
DRIVING UNDER THE INFLUENCE 4
SUSPICIOUS OCC 8
RECOVERED VEHICLE 7
DRUNKENNESS 1
TRESPASS 5
FRAUD 42
DISORDERLY CONDUCT 9
SEX OFFENSES, FORCIBLE 28
FORGERY/COUNTERFEITING 23
KIDNAPPING 9
EMBEZZLEMENT 17
STOLEN PROPERTY 8
LIQUOR LAWS 14
FAMILY OFFENSES 9
LOITERING 12
BAD CHECKS 2
TREA 1
GAMBLING 5
RUNAWAY 1
BRIBERY 4
PROSTITUTION 14
PORNOGRAPHY/OBSCENE MAT 3
SEX OFFENSES, NON FORCIBLE 4
SUICIDE 18
EXTORTION 2


In [13]:
types["BURGLARY"]

{'BURGLARY OF APARTMENT HOUSE, ATT FORCIBLE ENTRY',
 'BURGLARY OF APARTMENT HOUSE, FORCIBLE ENTRY',
 'BURGLARY OF APARTMENT HOUSE, UNLAWFUL ENTRY',
 'BURGLARY OF FLAT, ATTEMPTED FORCIBLE ENTRY',
 'BURGLARY OF FLAT, FORCIBLE ENTRY',
 'BURGLARY OF FLAT, UNLAWFUL ENTRY',
 'BURGLARY OF HOTEL ROOM, ATTEMPTED FORCIBLE ENTRY',
 'BURGLARY OF HOTEL ROOM, FORCIBLE ENTRY',
 'BURGLARY OF HOTEL ROOM, UNLAWFUL ENTRY',
 'BURGLARY OF RESIDENCE, ATTEMPTED FORCIBLE ENTRY',
 'BURGLARY OF RESIDENCE, FORCIBLE ENTRY',
 'BURGLARY OF RESIDENCE, UNLAWFUL ENTRY',
 'BURGLARY OF STORE, ATTEMPTED FORCIBLE ENTRY',
 'BURGLARY OF STORE, FORCIBLE ENTRY',
 'BURGLARY OF STORE, UNLAWFUL ENTRY',
 'BURGLARY OF WAREHOUSE, ATTEMPTED FORCIBLE ENTRY',
 'BURGLARY OF WAREHOUSE, FORCIBLE ENTRY',
 'BURGLARY OF WAREHOUSE, UNLAWFUL ENTRY',
 'BURGLARY, ATTEMPTED FORCIBLE ENTRY',
 'BURGLARY, FORCIBLE ENTRY',
 'BURGLARY, HOT PROWL, ATTEMPTED FORCIBLE ENTRY',
 'BURGLARY, HOT PROWL, FORCIBLE ENTRY',
 'BURGLARY, HOT PROWL, UNLAWFUL ENTRY'

Look at `PdDistrict`

In [14]:
rows = gen()
next(rows)
collections.Counter(row[6] for row in rows)

Counter({'': 1,
         'BAYVIEW': 212319,
         'CENTRAL': 213899,
         'INGLESIDE': 187014,
         'MISSION': 286559,
         'NORTHERN': 259985,
         'PARK': 120598,
         'RICHMOND': 111435,
         'SOUTHERN': 382295,
         'TARAVAL': 159729,
         'TENDERLOIN': 186210})

Look at `Resolution`

In [15]:
rows = gen()
next(rows)
collections.Counter(row[7] for row in rows)

Counter({'ARREST, BOOKED': 502931,
         'ARREST, CITED': 154759,
         'CLEARED-CONTACT JUVENILE FOR MORE INFO': 669,
         'COMPLAINANT REFUSES TO PROSECUTE': 8089,
         'DISTRICT ATTORNEY REFUSES TO PROSECUTE': 7955,
         'EXCEPTIONAL CLEARANCE': 4157,
         'JUVENILE ADMONISHED': 3004,
         'JUVENILE BOOKED': 13569,
         'JUVENILE CITED': 6586,
         'JUVENILE DIVERTED': 688,
         'LOCATED': 34461,
         'NONE': 1317932,
         'NOT PROSECUTED': 7717,
         'PROSECUTED BY OUTSIDE AGENCY': 5070,
         'PROSECUTED FOR LESSER OFFENSE': 84,
         'PSYCHOPATHIC CASE': 29181,
         'UNFOUNDED': 23192})

Look at `DayOfWeek` and `Date`.  These agree.

In [16]:
rows = gen()
next(rows)
for row in rows:
    dt = datetime.datetime.strptime(row[4], "%m/%d/%Y")
    assert dt.strftime("%A") == row[3]

Combine `Date` and `Time` and check these parse.

In [17]:
timestamps = []
rows = gen()
next(rows)
for row in rows:
    dt = datetime.datetime.strptime(row[4] + " " + row[5], "%m/%d/%Y %H:%M")
    timestamps.append(dt)
min(timestamps), max(timestamps)

(datetime.datetime(2003, 1, 1, 0, 1), datetime.datetime(2017, 9, 13, 23, 30))

Check `Location` is the same as `X` and `Y`.

In [18]:
rows = gen()
next(rows)
for row in rows:
    x, y = row[9], row[10]
    xx, yy = float(x), float(y)
    assert row[11] == "({}, {})".format(y, x)

Find the small number of "false" coordinates

In [19]:
rows = gen()
next(rows)
count = 0
for row in rows:
    x, y = row[9], row[10]
    xx, yy = float(x), float(y)
    if abs(yy - 90) < 1e-9:
        assert abs(xx + 120.5) < 1e-9
        count += 1
count

143

Look at repeats in `IncidntNum` (there are 326120) and check that the vast majority differ _only_ in `Category`,  `Descript` and/or `Resolution`.

In [20]:
rows = gen()
next(rows)
byid = collections.defaultdict(list)
for row in rows:
    byid[row[0]].append(row[1:])

In [21]:
byid = {k:v for k,v in byid.items() if len(v)>1}

In [22]:
unexpecteds = dict()
for key, value in byid.items():
    if not ( (len(set(tuple(row[2:6]) for row in value)) == 1) and
        (len(set(tuple(row[7:-1]) for row in value)) == 1) ):
        unexpecteds[key] = value

In [23]:
len(byid), len(unexpecteds)

(326120, 331)

The following is typical of the repeats

In [24]:
list(byid.items())[52]

('150100542',
 [['ASSAULT',
   'BATTERY',
   'Friday',
   '01/30/2015',
   '15:45',
   'TARAVAL',
   'NONE',
   '100 Block of APTOS AV',
   '-122.466758005159',
   '37.7291845796038',
   '(37.7291845796038, -122.466758005159)',
   '15010054204134'],
  ['SECONDARY CODES',
   'JUVENILE INVOLVED',
   'Friday',
   '01/30/2015',
   '15:45',
   'TARAVAL',
   'NONE',
   '100 Block of APTOS AV',
   '-122.466758005159',
   '37.7291845796038',
   '(37.7291845796038, -122.466758005159)',
   '15010054215500'],
  ['OTHER OFFENSES',
   'INCIDENT ON SCHOOL GROUNDS',
   'Friday',
   '01/30/2015',
   '15:45',
   'TARAVAL',
   'NONE',
   '100 Block of APTOS AV',
   '-122.466758005159',
   '37.7291845796038',
   '(37.7291845796038, -122.466758005159)',
   '15010054219089']])

The following is typical of the `unexpecteds`

In [25]:
list(unexpecteds.items())[7]

('150251214',
 [['VEHICLE THEFT',
   'STOLEN AUTOMOBILE',
   'Thursday',
   '03/19/2015',
   '13:00',
   'BAYVIEW',
   'ARREST, BOOKED',
   'BURROWS ST / BRUSSELS ST',
   '-122.406082132929',
   '37.728316332742',
   '(37.728316332742, -122.406082132929)',
   '15025121407021'],
  ['RECOVERED VEHICLE',
   'VEHICLE, RECOVERED, AUTO',
   'Monday',
   '03/23/2015',
   '09:30',
   'INGLESIDE',
   'ARREST, BOOKED',
   '400 Block of CORTLAND AV',
   '-122.417439996239',
   '37.7390953736683',
   '(37.7390953736683, -122.417439996239)',
   '15025121407041'],
  ['STOLEN PROPERTY',
   'STOLEN PROPERTY, POSSESSION WITH KNOWLEDGE, RECEIVING',
   'Thursday',
   '03/19/2015',
   '13:00',
   'BAYVIEW',
   'ARREST, BOOKED',
   'BURROWS ST / BRUSSELS ST',
   '-122.406082132929',
   '37.728316332742',
   '(37.728316332742, -122.406082132929)',
   '15025121411012'],
  ['OTHER OFFENSES',
   'TRAFFIC COLLISION, HIT & RUN, PROPERTY DAMAGE',
   'Thursday',
   '03/19/2015',
   '13:00',
   'BAYVIEW',
   'ARRES

## Look at the `address` field.

Always of the form:

- `200 Block of ...`
- `... / ...`

where "..." is a street name.

In [26]:
rows = gen()
next(rows)
addresses = set(row[8] for row in rows)
len(addresses)

25093

In [27]:
list(addresses)[:5]

['ONONDAGA AV / ALEMANY BL',
 '3100 Block of FILLMORE ST',
 'SHIPLEY ST / 6TH ST',
 '500 Block of DUBOCE AV',
 '0 Block of 10TH ST']

In [28]:
unexpected = []
streets = set()

for address in addresses:
    if " Block of " in address:
        i = address.index(" Block of ")
        num = int(address[:i])
        assert num % 100 == 0
        streets.add(address[i+10:])
        continue
    if " / " in address:
        assert len(address.split(" / ")) == 2
        for p in address.split(" / "):
            streets.add(p)
        continue
    unexpected.append(address)
unexpected

['GENEVA AV /', 'TEHAMA ST /', 'CLEMENTINA ST /']

In [29]:
streets = list(streets)
streets.sort()
len(streets)

2139